# AUTO$^2$ and a "high-dimensional" example : The Reinhold & Pierrehumbert 1982 model

**This is an example on how to use AUTO$^2$ to compute bifurcations and solutions in the case of a rather high-dimensional model (from the point of view of bifurcation theory). This example aims to provide a glimpse at how to use AUTO$^2$ to deal with such system.**

This example is concerned with with a 2-layer channel QG atmosphere truncated at wavenumber 2 on a beta-plane and a simple orography (a mountain and a valley), derived in 1982 by Reinhold and Pierrehumbert (RP1982 hereafter).

![RP1982Url](https://github.com/Climdyn/qgs/blob/master/misc/figs/readme.gif?raw=true)
Source: [qgs code repository](https://github.com/Climdyn/qgs)

The AUTO Fortran and configuration files have been prepared thanks to the qgs framework.
See in particular [this notebook](https://github.com/Climdyn/qgs/blob/master/notebooks/symbolic_outputs/symbolic_output_land_atmosphere-AUTO.ipynb) for an example on how to prepare those files.

The equations of the model are too complex to be displayed here, please see the original article for details.
See also [here](https://github.com/Climdyn/qgs/blob/master/notebooks/introduction_qgs.ipynb) for an introduction to the `qgs` framework involving the RP1982 model.
However, the model dimension is 20, and there are two sets of variables, $\psi_{\rm a, i}$ and $\theta_{\rm a, i}$, corresponding to the spectral components of respectively the barotropic and baroclinic streamfunctions. These two streamfunctions are respectively related themselves to the 500hPa pressure and temperature anomalies.

In addition, five **nondimensional** parameters have been left as variables in the model tendencies equations:

* $\theta^\ast_1$ : The first spectral components of the equilibrium temperature profile at 500hPa (the middle of the model atmosphere). It corresponds to a zonal, North-South, temperature gradient.
* $h_{k,2}$ : The second spectral components of the orography, corresponding to a mountain at the zonal border of the domain, and a valley in the center. It's value determine the relative height between the top of the mountain, and the bottom of the valley.
* $k_d$ : The atmosphere bottom friction coefficient with the ground.
* $k'_d$ : The atmosphere internal friction coefficient.
* $\sigma$ : The static stability of the atmosphere.

The free parameter in the current example notebook is $\theta^\ast_1$ (`thetas_1`) . Other parameters are fixed.

We are thus going to find the fixed points and periodic orbits of this system and continue them by varying $\theta^\ast_1$.

#### References

* Reinhold, B. B., & Pierrehumbert, R. T. (1982). *Dynamics of weather regimes: Quasi-stationary waves and blocking.* Monthly Weather Review, **110** (9), 1105-1145. [doi:10.1175/1520-0493(1982)110%3C1105:DOWRQS%3E2.0.CO;2](https://doi.org/10.1175/1520-0493(1982)110%3C1105:DOWRQS%3E2.0.CO;2)

## Code

First we set the Python path if needed:

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../../')])

And load the needed libraries, including AUTO$^2$:

In [ ]:
import numpy as np
from numba import njit
from scipy.optimize import root
from scipy.integrate import solve_ivp

In [ ]:
from auto2.diagrams.bifurcations import BifurcationDiagram

Loading also `qgs` framework to generate the model output:

In [ ]:
# Installing qgs if needed (uncomment the line below and run)
# !pip install qgs

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

Setting the random number generator seed for reproducibility (comment if you don't need that)


In [ ]:
np.random.seed(1402)

Creating the model equations

In [ ]:
# Model parameters instantiation with some non-default specs
model_parameters = QgParams({'phi0_npi': np.deg2rad(50.)/np.pi, 'hd':0.045})
# Mode truncation at the wavenumber 2 in both x and y spatial coordinate
model_parameters.set_atmospheric_channel_fourier_modes(2, 2)

# Setting the orography depth and the meridional temperature gradient
model_parameters.ground_params.set_orography(0.2, 1)
model_parameters.atemperature_params.set_thetas(0.1, 0)

In [ ]:
f, Df = create_tendencies(model_parameters)

In [ ]:
# defining a tendencies function with a f(x) signature
@njit
def fnt(x):
    return f(0.,x)

and define a set of standard parameters:

In [ ]:
params = {
    'thetas_1': 0.1,
    'hk_2': 0.2,
    'k_d': 0.1,
    'k_p': 0.01,
    'sigma': 0.2,
}


For reference later, we can compute a long trajectory on the attractor of this model for $\theta^\ast_1 = 0.1$:

In [ ]:
#first a transient
ic = np.zeros(model_parameters.ndim) + 0.0001
transient = solve_ivp(f, (0., 200000.), ic)

In [ ]:
# then the trajectory itself
ic = transient['y'][:, -1]
trajectory = solve_ivp(f, (0., 100000.), ic)

Finding all the fixed points of system for $\theta^\ast_1 = 0.1$ :

In [ ]:
nsearch = 10000

# Start on random initial conditions
ic = 2 * (np.random.rand(nsearch, model_parameters.ndim) - 0.5) * 0.1

eps = 1.e-6
fixed_points = dict()

sol_idx = 1
for i in range(nsearch):
    sol = root(fnt, ic[i, :])
    if sol.success:
        for idx in fixed_points:
            if np.linalg.norm(fixed_points[idx] - sol.x) < eps:
                break
        else:
            fixed_points[sol_idx] = sol.x
            sol_idx+=1


We have now the list of fixed points `fixed_points` and parameters dictionnary `params` that AUTO$^2$ will have to continue

In [ ]:
initial_points = list()

for p in fixed_points:
    initial_points.append({'parameters': params, 'initial_data': fixed_points[p]})


and thus we are now ready to compute the diagram of fixed points as a function of $\theta^\ast_1$ (`thetas_1`). Note that we specify that the bifurcation diagram object must load the ̀`qgs_RP1982.f90` and `c.qgs_RP1982` files where the RP 1982 model equations and continuation parameters have been written:

In [ ]:
b = BifurcationDiagram('qgs_RP1982')

b.compute_fixed_points_diagram(initial_points,
                               extra_comparison_parameters=['psi_a_1', 'psi_a_2', 'psi_a_5'], 
                               comparison_tol=[1.e-4] * 4,
                               ICP=['thetas_1'], 
                               NPR=0)

We can now plot the result as functions of $\theta^\ast_1$ (`thetas_1`) and $L^2$ norm :

In [ ]:
b.plot_fixed_points_diagram();

and also as functions of $\psi_{\rm a,1}$ and $\psi_{\rm a,2}$ :

In [ ]:
b.plot_fixed_points_diagram((2,3));

or in 3D as functions of $\theta^\ast_1$ (`thetas_1`), $L^2$ norm and $\psi_{\rm a,1}$ :

In [ ]:
b.plot_fixed_points_diagram_3D();

We see that at many branches were found, and many Hopf bifurcations. 

We can continue periodic orbits out of these Hopf bifurcations, stopping at the 3rd generation of found bifurcation points (level 3) : 

**Beware: starting the next cell will launch computations that last 20 minutes on a recent 12 cores CPU !**

In [ ]:
b.compute_periodic_orbits_diagram(
    3, 
    max_number_bp=None, 
    maximum_period=1.e9,  # avoid continuity issues when the period get too big
    extra_comparison_parameters=['psi_a_1', 'psi_a_2', 'psi_a_5'], 
    comparison_tol=[1.e-4] * 4,
    ICP=['thetas_1'], 
    NMX=1000,  # cutoff to avoid too long computations in this example, we might miss some solutions though  
)


and plot the results on a (complicated) bifurcation diagram:

In [ ]:
ax = b.plot_fixed_points_diagram(legend=False)
b.plot_periodic_orbits_diagram(ax=ax, cmap='gist_ncar', legend=False);

We can also plot both the bifurcation diagram and the solutions for a given value of $\theta^\ast_1$:

In [ ]:
b.plot_diagram_and_solutions(0.11, solutions_variables=(2, 1), fixed_points_diagram_kwargs={'legend': False}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar', 'legend': False});

or plot a single branch and its solutions:

In [ ]:
# plotting branch 21
b.plot_single_po_branch_and_solutions(21, solutions_variables=(2, 1), cmap='Blues_r')

Finally, it is not hard to also plot the dynamics on the attractor (represented by the long trajectory computed beforehand) on top of the solutions to see their relevance:

In [ ]:
axs = b.plot_diagram_and_solutions(0.1, solutions_variables=(2, 1), fixed_points_diagram_kwargs={'legend': False}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar', 'legend': False})
axs[1].plot(trajectory['y'][2], trajectory['y'][1], marker='o', ms=0.17, ls='', color='darkgray');

One can see that the chaotic dynamics is roughly explained by the found periodic orbits (although many many more are still missing).

You can restart the computation up to a higher level by uncommenting and running the code below:

In [ ]:
# b.compute_periodic_orbits_diagram(
#     5, 
#     max_number_bp=None, 
#     maximum_period=1.e9,  # avoid continuity issues when the period get too big
#     extra_comparison_parameters=['psi_a_1', 'psi_a_2', 'psi_a_5'], 
#     comparison_tol=[1.e-4] * 4,
#     ICP=['thetas_1'], 
#     NMX=1000,  # cutoff to avoid too long computations in this example, we might miss some solutions though  
# )